In [2]:
"""
╔════════════════════════════════════════════════════════════════════════════════╗
║                                                                                ║
║               ALGORITHMIC PREDATORY EQUILIBRIUM (APE)                          ║
║            Simulation Code Repository - Complete Implementation                ║
║                                                                                ║
║  Title: Algorithmic Predatory Equilibrium: Decentralized Markets Under         ║
║         Regime Shift from Speed to Cognition                                   ║
║                                                                                ║
║  Author: [Your Name]                                                           ║
║  Date: January 2026                                                            ║
║  Status: Publication-Ready                                                     ║
║                                                                                ║
║  This module contains all agent-based simulation code used to validate         ║
║  Propositions 1 and 2, and test regulatory effectiveness (Scenario A, B, C).   ║
║                                                                                ║
╚════════════════════════════════════════════════════════════════════════════════╝

SIMULATION STRUCTURE:
====================

1. CONFIG & PARAMETERS
   - Behavioral calibration (λ = 2.25)
   - Market parameters (reserves, prices, gas costs)
   - Agent populations (5 predators, 95 prey)
   - Episode/step configuration (100 episodes × 250 steps)

2. CORE CLASSES
   - BaseAgent (abstract agent class)
   - PredatorAgent (MARL-based predator with policy gradient)
   - PreyAgent (behavioral trader with loss aversion)
   - Market (AMM microstructure)
   - Simulator (orchestrates multi-scenario runs)

3. UTILITY FUNCTIONS
   - Prospect theory calculations (utility, panic function)
   - Data logging and aggregation
   - Statistical validation

4. EXECUTION
   - Run three scenarios (A: Rational, B: Predatory, C: Regulated)
   - Generate results table and figure

════════════════════════════════════════════════════════════════════════════════
"""

import numpy as np
import pandas as pd
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional
from enum import Enum
from scipy.special import expit as sigmoid  # logistic sigmoid
import json
from datetime import datetime


# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 1: CONFIGURATION & CALIBRATION PARAMETERS
# ═══════════════════════════════════════════════════════════════════════════════

@dataclass
class SimulationConfig:
    """
    Master configuration for all simulation scenarios.

    Attributes match empirical calibrations from:
    - Kaustia & Knüpfer (2014): Loss aversion λ = 2.25
    - Lussange et al. (2024): MARL on crypto markets
    - Flashbots (2024-2025): MEV empirical data
    """

    # Market Structure
    initial_reserves: float = 1000.0          # AMM liquidity (normalized)
    initial_price: float = 100.0              # Initial token price
    initial_balance_predator: float = 500.0   # Predator capital
    initial_balance_prey: float = 200.0       # Prey capital

    # Behavioral Parameters
    loss_aversion_lambda: float = 2.25        # Prospect theory (meta-analyzed)
    panic_multiplier: float = 3.0             # Extreme loss reaction
    cascade_threshold: float = 0.30           # >30% panic triggers cascade

    # Agent Population
    n_predators: int = 5                      # MARL agents
    n_prey: int = 95                          # Behavioral traders
    n_liquidity_providers: int = 0            # Passive LPs (if needed)

    # Episode & Step Configuration
    n_episodes: int = 100                     # 100 trading days
    steps_per_episode: int = 250              # 250 steps = 4 hours/day

    # Cost Parameters
    gas_cost_per_trade: float = 0.50          # Transaction cost
    detection_probability: float = 0.02       # 2% enforcement detection

    # Regulatory Parameters (Scenario C)
    cancellation_threshold: float = 0.90      # 90% cancellation = violation
    fine_magnitude: float = 100.0             # $100 per violation

    # Random Seed
    random_seed: int = 42


@dataclass
class EpisodeMetrics:
    """Container for episode-level statistics."""

    episode_number: int
    predator_profit: float = 0.0
    predator_volume: float = 0.0
    market_volatility: float = 0.0
    prey_loss: float = 0.0
    prey_volume: float = 0.0
    cascade_events: int = 0
    panic_selling_count: int = 0
    violations_detected: int = 0
    regulation_fines: float = 0.0


# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 2: BEHAVIORAL FINANCE UTILITY FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════════════

class ProspectTheory:
    """
    Prospect theory implementation for behavioral agents.

    Reference: Kahneman & Tversky (1979); Tversky & Kahneman (1992)

    Utility function:
        U(w) = w^α                if w ≥ 0
        U(w) = -λ(-w)^β           if w < 0

    Empirically calibrated: α ≈ β ≈ 0.88; λ ∈ [1.8, 2.1]
    """

    ALPHA = 0.88        # Gain exponent
    BETA = 0.88         # Loss exponent
    REFERENCE_POINT = 0 # Neutral wealth level

    @staticmethod
    def utility(wealth: float, lambda_param: float = 2.25) -> float:
        """
        Compute prospect theory utility.

        Args:
            wealth: Current wealth level
            lambda_param: Loss aversion coefficient

        Returns:
            Utility value
        """
        if wealth >= ProspectTheory.REFERENCE_POINT:
            return wealth ** ProspectTheory.ALPHA
        else:
            loss_magnitude = abs(wealth - ProspectTheory.REFERENCE_POINT)
            return -lambda_param * (loss_magnitude ** ProspectTheory.BETA)

    @staticmethod
    def panic_threshold(
        price_shock: float,
        market_volatility: float,
        lambda_param: float = 2.25
    ) -> float:
        """
        Compute probability of panic selling given shock.

        Panic function: P(sell | shock) = sigmoid(λ * normalized_shock)

        Where normalized_shock = (P_t-1 - P_t) / σ_t

        Args:
            price_shock: Absolute price change (P_t-1 - P_t)
            market_volatility: Current σ_t (volatility)
            lambda_param: Loss aversion (amplification factor)

        Returns:
            Probability ∈ [0, 1]
        """
        if market_volatility < 1e-6:  # Avoid division by zero
            market_volatility = 1e-6

        normalized_shock = price_shock / market_volatility
        panic_probability = sigmoid(lambda_param * normalized_shock)

        return panic_probability


# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 3: AGENT CLASSES
# ═══════════════════════════════════════════════════════════════════════════════

class BaseAgent:
    """
    Abstract base class for all agents.

    Attributes:
        agent_id: Unique identifier
        balance: Current capital/token holdings
        wealth_history: Time series of wealth
        is_predator: Boolean flag for agent type
    """

    def __init__(self, agent_id: int, initial_balance: float, is_predator: bool):
        self.agent_id = agent_id
        self.balance = initial_balance
        self.wealth_history = [initial_balance]
        self.is_predator = is_predator
        self.trades_executed = 0
        self.total_volume = 0.0

    def get_current_wealth(self) -> float:
        """Return current balance."""
        return self.balance

    def add_to_wealth_history(self, new_balance: float):
        """Append to wealth history."""
        self.balance = new_balance
        self.wealth_history.append(new_balance)

    def execute_trade(self, amount: float, direction: str):
        """
        Record a trade execution.

        Args:
            amount: Trade size (absolute)
            direction: 'buy' or 'sell'
        """
        self.trades_executed += 1
        self.total_volume += abs(amount)


class PredatorAgent(BaseAgent):
    """
    MARL-based predator agent that learns volatility induction.

    Uses Policy Gradient optimization (simplified REINFORCE).
    State: (price, order book, mempool visibility)
    Action: Market order size ∈ [-max_order, +max_order]
    Reward: Profit from exploitation

    Key insight: Without behavioral prey, predator profit is minimal.
                 With behavioral prey (λ=2.25), predator discovers
                 volatility-inducing strategies as optimal.
    """

    def __init__(
        self,
        agent_id: int,
        initial_balance: float,
        config: SimulationConfig
    ):
        super().__init__(agent_id, initial_balance, is_predator=True)
        self.config = config

        # Policy parameters (simplified MARL)
        self.learning_rate = 0.01
        self.policy_weights = np.array([0.1, 0.2, 0.3])  # [size, timing, direction]

        # Reward history for policy gradient
        self.episode_rewards = []
        self.episode_actions = []

    def decide_action(
        self,
        current_price: float,
        order_book_imbalance: float,
        prey_panic_level: float,
        market_volatility: float
    ) -> Tuple[float, str]:
        """
        Compute action via learned policy.

        Policy discovery: MARL learns that inducing volatility
        triggers panic selling in behavioral prey (λ=2.25).

        Action = (order_size, direction)

        Args:
            current_price: Current spot price
            order_book_imbalance: OB_buy - OB_sell (signed)
            prey_panic_level: Fraction of prey in panic state
            market_volatility: Current σ_t

        Returns:
            Tuple of (order_size, direction)
        """
        # Policy: exploit panic via market impact
        # If prey_panic_level high AND volatility rising, place large market order

        state = np.array([
            order_book_imbalance,
            prey_panic_level,
            market_volatility
        ])

        # Simplified policy (linear): π(a|s) ∝ weights · state
        action_signal = np.dot(self.policy_weights, state)

        # Map to order size (sigmoid bounded)
        max_order = 10.0
        order_size = max_order * sigmoid(action_signal)

        # Direction: exploit if panic high, otherwise accumulate
        direction = 'sell' if prey_panic_level > 0.2 else 'buy'

        return order_size, direction

    def update_policy(self, episode_return: float):
        """
        Simple policy gradient update (REINFORCE).

        In production: Would use PPO, TRPO, A3C.
        For simulation: Simplified update proportional to returns.
        """
        if episode_return > 0:
            # Positive return: increase aggressiveness
            self.policy_weights *= (1.0 + self.learning_rate * 0.1)
            self.policy_weights = np.clip(self.policy_weights, 0.01, 1.0)


class PreyAgent(BaseAgent):
    """
    Behavioral trader with loss aversion (λ = 2.25).

    Uses prospect theory for decision-making.
    Exhibits panic selling when losses exceed pain threshold.

    Key mechanism: Sigmoid coupling creates threshold effect.
    Small shocks → minimal selling. Large shocks → massive panic.
    MARL learns to induce large shocks.
    """

    def __init__(
        self,
        agent_id: int,
        initial_balance: float,
        config: SimulationConfig
    ):
        super().__init__(agent_id, initial_balance, is_predator=False)
        self.config = config
        self.reference_price = initial_balance / 100  # Reference for loss calculation
        self.is_panicking = False
        self.panic_threshold_triggered = False

    def decide_action(
        self,
        current_price: float,
        price_change: float,
        market_volatility: float,
        trading_probability: float = 0.15
    ) -> Tuple[bool, float, str]:
        """
        Behavioral decision via prospect theory.

        Decision tree:
        1. Base trading probability: 15% of prey trade per step
        2. If loss: amplify by sigmoid(λ * normalized_shock)
        3. If cascade: panic multiplier 3× (herd behavior)

        Args:
            current_price: Current token price
            price_change: P_t-1 - P_t (negative = loss)
            market_volatility: Current σ_t
            trading_probability: Base trading frequency

        Returns:
            Tuple of (will_trade, order_size, direction)
        """
        # Base trading decision
        will_trade = np.random.random() < trading_probability

        if not will_trade:
            return False, 0.0, 'hold'

        # Loss aversion amplification
        if price_change > 0:  # Loss occurred (price fell)
            panic_prob = ProspectTheory.panic_threshold(
                price_shock=price_change,
                market_volatility=market_volatility,
                lambda_param=self.config.loss_aversion_lambda
            )

            # Threshold effect: if panic probable, sell with amplified size
            if np.random.random() < panic_prob:
                self.is_panicking = True
                order_size = 5.0 * self.config.panic_multiplier  # Amplified
                direction = 'sell'
            else:
                self.is_panicking = False
                order_size = 2.0  # Normal trade size
                direction = 'buy'  # Default to buy after loss (averaging down)
        else:
            # Gain: conservative trading
            self.is_panicking = False
            order_size = 1.5
            direction = 'buy'

        return True, order_size, direction


# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 4: MARKET MODEL (AMM MICROSTRUCTURE)
# ═══════════════════════════════════════════════════════════════════════════════

class Market:
    """
    Automated Market Maker (Uniswap V2 style).

    Constant product formula: x * y = k

    Predators see full mempool (information advantage).
    Prey see only confirmed trades.
    """

    def __init__(self, config: SimulationConfig):
        self.config = config
        self.reserve_token = config.initial_reserves
        self.reserve_stable = config.initial_reserves * config.initial_price
        self.k = self.reserve_token * self.reserve_stable  # Constant product

        # Price history
        self.price_history = [config.initial_price]
        self.volatility_history = [0.0]

        # Order book simulation
        self.pending_orders = []  # Mempool
        self.executed_trades = []

    def get_current_price(self) -> float:
        """Compute current spot price from reserves."""
        return self.reserve_stable / self.reserve_token

    def execute_trade(
        self,
        order_size: float,
        direction: str,
        agent_id: int
    ) -> Tuple[float, float]:
        """
        Execute market order via AMM.

        Constant product: (x + dx) * (y - dy) = x * y
        Price impact: larger orders face worse prices.

        Args:
            order_size: Token quantity
            direction: 'buy' or 'sell'
            agent_id: Executing agent ID

        Returns:
            Tuple of (executed_price, cost_paid)
        """
        if direction == 'buy':
            # Swap stable for tokens
            # Need to compute: given dy_out (order_size), what is dx_in?
            # Formula: dx = (k / (y - dy_out)) - x

            min_reserve = 1e-6  # Prevent zero
            new_reserve_stable = self.k / (self.reserve_token + order_size)
            dx = max(self.reserve_stable - new_reserve_stable, min_reserve)

            executed_price = dx / order_size

            # Update reserves
            self.reserve_token += order_size
            self.reserve_stable -= dx

            cost = dx

        else:  # sell
            # Swap tokens for stable
            # Given dx_in (order_size), compute dy_out
            # Formula: dy = y - (k / (x + dx))

            min_reserve = 1e-6
            new_reserve_token = self.k / (self.reserve_stable + order_size)
            dy = max(self.reserve_token - new_reserve_token, min_reserve)

            executed_price = dy / order_size

            # Update reserves
            self.reserve_token -= dy
            self.reserve_stable += order_size

            cost = -dy  # Negative because we receive stable

        # Record trade
        self.executed_trades.append({
            'agent_id': agent_id,
            'direction': direction,
            'size': order_size,
            'price': executed_price,
            'timestamp': len(self.executed_trades)
        })

        return executed_price, cost

    def update_price_history(self):
        """Update price and volatility tracking."""
        current_price = self.get_current_price()
        self.price_history.append(current_price)

        # Compute rolling volatility (last 20 steps)
        if len(self.price_history) > 1:
            returns = np.diff(np.log(self.price_history[-20:]))
            volatility = np.std(returns) if len(returns) > 0 else 0.0
        else:
            volatility = 0.0

        self.volatility_history.append(volatility)

        return current_price, volatility


# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 5: SIMULATION ENGINE
# ═══════════════════════════════════════════════════════════════════════════════

class SimulationEngine:
    """
    Orchestrates agent-based simulation across episodes.

    Three scenarios:
    - A: Rational agents only (baseline)
    - B: MARL predators + behavioral prey (APE)
    - C: Scenario B + regulatory enforcement

    Execution:
    - 100 episodes (trading days)
    - 250 steps per episode (1-minute resolution, ~4 hours/day)
    - Tracks: profits, volatility, cascades, regulatory violations
    """

    def __init__(self, config: SimulationConfig):
        self.config = config
        np.random.seed(config.random_seed)

        self.results = {
            'A': [],  # Scenario A results
            'B': [],  # Scenario B results
            'C': []   # Scenario C results
        }

    def initialize_agents(self, include_predators: bool = True):
        """
        Create agent population.

        Args:
            include_predators: If False, scenario A (rational only)

        Returns:
            Tuple of (predators, prey)
        """
        predators = []
        prey = []

        if include_predators:
            for i in range(self.config.n_predators):
                predators.append(
                    PredatorAgent(i, self.config.initial_balance_predator, self.config)
                )

        for i in range(self.config.n_prey):
            prey.append(
                PreyAgent(
                    self.config.n_predators + i,
                    self.config.initial_balance_prey,
                    self.config
                )
            )

        return predators, prey

    def run_episode(
        self,
        episode_num: int,
        predators: List[PredatorAgent],
        prey: List[PreyAgent],
        market: Market,
        scenario: str = 'B'
    ) -> EpisodeMetrics:
        """
        Execute one trading day (250 steps = 4 hours).

        Args:
            episode_num: Episode number
            predators: List of predator agents
            prey: List of prey agents
            market: Market instance
            scenario: 'A', 'B', or 'C'

        Returns:
            EpisodeMetrics with statistics
        """
        metrics = EpisodeMetrics(episode_number=episode_num)

        initial_predator_wealth = sum(p.get_current_wealth() for p in predators) if predators else 0.0
        initial_prey_wealth = sum(p.get_current_wealth() for p in prey)

        # Simulate 250 steps
        for step in range(self.config.steps_per_episode):

            # PREDATOR DECISIONS
            for predator in predators:
                current_price = market.get_current_price()

                # Compute state variables
                order_book_imbalance = np.random.normal(0, 1)  # Simplified
                prey_panic_level = np.mean([
                    1.0 if p.is_panicking else 0.0 for p in prey
                ])
                market_volatility = market.volatility_history[-1] if market.volatility_history else 0.01

                # Get action from policy
                order_size, direction = predator.decide_action(
                    current_price,
                    order_book_imbalance,
                    prey_panic_level,
                    market_volatility
                )

                # Execute
                executed_price, cost = market.execute_trade(order_size, direction, predator.agent_id)

                # Update predator balance
                if direction == 'buy':
                    predator.balance -= cost
                else:
                    predator.balance += cost

                predator.execute_trade(order_size, direction)
                metrics.predator_volume += order_size

            # PREY DECISIONS
            cascade_triggered = False
            panic_count = 0

            for prey_agent in prey:
                current_price = market.get_current_price()
                price_change = market.price_history[-2] - current_price if len(market.price_history) > 1 else 0.0
                market_volatility = market.volatility_history[-1] if market.volatility_history else 0.01

                will_trade, order_size, direction = prey_agent.decide_action(
                    current_price,
                    price_change,
                    market_volatility
                )

                if will_trade:
                    executed_price, cost = market.execute_trade(order_size, direction, prey_agent.agent_id)

                    if direction == 'buy':
                        prey_agent.balance -= cost
                    else:
                        prey_agent.balance += cost

                    prey_agent.execute_trade(order_size, direction)
                    metrics.prey_volume += order_size

                if prey_agent.is_panicking:
                    panic_count += 1

            # CASCADE DETECTION
            panic_fraction = panic_count / len(prey)
            if panic_fraction > self.config.cascade_threshold:
                cascade_triggered = True
                metrics.cascade_events += 1

            metrics.panic_selling_count = panic_count

            # UPDATE MARKET STATE
            current_price, volatility = market.update_price_history()

            # REGULATORY ENFORCEMENT (Scenario C)
            if scenario == 'C':
                # Fine predators for suspicious behavior (high cancellation rate)
                cancellation_rate = np.random.uniform(0.5, 0.95)  # Predators try to evade

                if cancellation_rate > self.config.cancellation_threshold:
                    metrics.violations_detected += 1
                    metrics.regulation_fines += self.config.fine_magnitude

                    # Apply fines to predators
                    if predators:
                        fine_per_predator = self.config.fine_magnitude / len(predators)
                        for predator in predators:
                            predator.balance -= fine_per_predator

        # END-OF-EPISODE CALCULATION
        final_predator_wealth = sum(p.get_current_wealth() for p in predators) if predators else 0.0
        final_prey_wealth = sum(p.get_current_wealth() for p in prey)

        metrics.predator_profit = final_predator_wealth - initial_predator_wealth
        metrics.prey_loss = -(final_prey_wealth - initial_prey_wealth)
        metrics.market_volatility = np.mean(market.volatility_history[-50:]) if len(market.volatility_history) > 50 else 0.0

        return metrics

    def run_scenario(self, scenario: str) -> List[EpisodeMetrics]:
        """
        Run complete scenario (100 episodes).

        Args:
            scenario: 'A' (rational), 'B' (predatory), or 'C' (regulated)

        Returns:
            List of EpisodeMetrics
        """
        print(f"\n{'='*80}")
        print(f"Running Scenario {scenario}")
        print(f"{'='*80}")

        include_predators = (scenario in ['B', 'C'])
        predators, prey = self.initialize_agents(include_predators)
        market = Market(self.config)

        episode_results = []

        for episode in range(self.config.n_episodes):
            metrics = self.run_episode(
                episode,
                predators,
                prey,
                market,
                scenario=scenario
            )

            episode_results.append(metrics)

            if (episode + 1) % 10 == 0:
                avg_predator_profit = np.mean([
                    m.predator_profit for m in episode_results[-10:]
                ])
                avg_volatility = np.mean([
                    m.market_volatility for m in episode_results[-10:]
                ])
                print(f"Episode {episode+1:3d} | "
                      f"Predator Profit: ${avg_predator_profit:8.2f} | "
                      f"Volatility: {avg_volatility:6.2%}")

        self.results[scenario] = episode_results
        return episode_results


# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 6: ANALYSIS & RESULTS AGGREGATION
# ═══════════════════════════════════════════════════════════════════════════════

class ResultsAnalyzer:
    """
    Analyze simulation results across scenarios.
    Compute statistics, generate tables, validate propositions.
    """

    def __init__(self, simulator: SimulationEngine):
        self.simulator = simulator

    def summarize_scenario(self, scenario: str) -> Dict:
        """
        Compute summary statistics for scenario.

        Args:
            scenario: 'A', 'B', or 'C'

        Returns:
            Dictionary of aggregate metrics
        """
        results = self.simulator.results[scenario]

        predator_profits = [m.predator_profit for m in results]
        prey_losses = [m.prey_loss for m in results]
        volatilities = [m.market_volatility for m in results]
        cascades = [m.cascade_events for m in results]

        return {
            'scenario': scenario,
            'mean_predator_daily_profit': np.mean(predator_profits),
            'total_predator_profit': np.sum(predator_profits),
            'mean_prey_daily_loss': np.mean(prey_losses),
            'total_prey_loss': np.sum(prey_losses),
            'mean_volatility': np.mean(volatilities),
            'total_cascades': np.sum(cascades),
            'median_cascades_per_episode': np.median(cascades),
        }

    def generate_results_table(self) -> pd.DataFrame:
        """
        Generate comparison table across all scenarios.

        Returns:
            DataFrame with results
        """
        summaries = {
            scenario: self.summarize_scenario(scenario)
            for scenario in ['A', 'B', 'C']
        }

        df = pd.DataFrame({
            'Scenario A (Rational)': {
                'Predator Daily Profit': summaries['A']['mean_predator_daily_profit'],
                'Predator Total Profit': summaries['A']['total_predator_profit'],
                'Prey Daily Loss': summaries['A']['mean_prey_daily_loss'],
                'Prey Total Loss': summaries['A']['total_prey_loss'],
                'Mean Volatility': summaries['A']['mean_volatility'],
                'Total Cascades': summaries['A']['total_cascades'],
            },
            'Scenario B (Predatory)': {
                'Predator Daily Profit': summaries['B']['mean_predator_daily_profit'],
                'Predator Total Profit': summaries['B']['total_predator_profit'],
                'Prey Daily Loss': summaries['B']['mean_prey_daily_loss'],
                'Prey Total Loss': summaries['B']['total_prey_loss'],
                'Mean Volatility': summaries['B']['mean_volatility'],
                'Total Cascades': summaries['B']['total_cascades'],
            },
            'Scenario C (Regulated)': {
                'Predator Daily Profit': summaries['C']['mean_predator_daily_profit'],
                'Predator Total Profit': summaries['C']['total_predator_profit'],
                'Prey Daily Loss': summaries['C']['mean_prey_daily_loss'],
                'Prey Total Loss': summaries['C']['total_prey_loss'],
                'Mean Volatility': summaries['C']['mean_volatility'],
                'Total Cascades': summaries['C']['total_cascades'],
            }
        })

        return df

    def validate_propositions(self) -> Dict:
        """
        Check Proposition 1 & 2 hypotheses.

        Returns:
            Dictionary of validation results
        """
        summary_a = self.summarize_scenario('A')
        summary_b = self.summarize_scenario('B')
        summary_c = self.summarize_scenario('C')

        # Proposition 1: Volatility Induction
        volatility_increase = (summary_b['mean_volatility'] - summary_a['mean_volatility']) / summary_a['mean_volatility']
        alpha_increase = (summary_b['mean_predator_daily_profit'] - summary_a['mean_predator_daily_profit']) / summary_a['mean_predator_daily_profit']

        # Proposition 2: Information Cascades
        cascade_increase = (summary_b['total_cascades'] - summary_a['total_cascades']) / (summary_a['total_cascades'] + 1)

        # Regulatory Effectiveness
        profit_reduction = (summary_b['mean_predator_daily_profit'] - summary_c['mean_predator_daily_profit']) / summary_b['mean_predator_daily_profit']
        cascade_reduction = (summary_b['total_cascades'] - summary_c['total_cascades']) / (summary_b['total_cascades'] + 1)

        return {
            'proposition_1': {
                'volatility_increase_pct': volatility_increase * 100,
                'alpha_increase_pct': alpha_increase * 100,
                'validated': volatility_increase > 1.0 and alpha_increase > 10.0
            },
            'proposition_2': {
                'cascade_increase_pct': cascade_increase * 100,
                'validated': cascade_increase > 10.0
            },
            'regulatory_gap': {
                'profit_reduction_pct': profit_reduction * 100,
                'cascade_reduction_pct': cascade_reduction * 100,
                'residual_alpha_daily': summary_c['mean_predator_daily_profit'],
                'insufficient_enforcement': summary_c['mean_predator_daily_profit'] > 0.0
            }
        }


# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 7: MAIN EXECUTION
# ═══════════════════════════════════════════════════════════════════════════════

def main():
    """
    Execute complete simulation suite.

    1. Initialize configuration
    2. Run three scenarios (A, B, C)
    3. Analyze results
    4. Validate propositions
    5. Output results
    """

    print("\n" + "="*80)
    print("ALGORITHMIC PREDATORY EQUILIBRIUM (APE) SIMULATION")
    print("Publication-Ready Code")
    print("="*80)

    # CONFIGURATION
    config = SimulationConfig(
        n_episodes=100,
        steps_per_episode=250,
        loss_aversion_lambda=2.25,
        random_seed=42
    )

    print(f"\nConfiguration:")
    print(f"  Episodes: {config.n_episodes}")
    print(f"  Steps/Episode: {config.steps_per_episode}")
    print(f"  Loss Aversion (λ): {config.loss_aversion_lambda}")
    print(f"  Predators: {config.n_predators}")
    print(f"  Prey: {config.n_prey}")

    # SIMULATION ENGINE
    engine = SimulationEngine(config)

    # RUN SCENARIOS
    engine.run_scenario('A')
    engine.run_scenario('B')
    engine.run_scenario('C')

    # ANALYSIS
    analyzer = ResultsAnalyzer(engine)

    print("\n" + "="*80)
    print("RESULTS SUMMARY")
    print("="*80)

    results_table = analyzer.generate_results_table()
    print("\n" + results_table.to_string())

    print("\n" + "="*80)
    print("PROPOSITION VALIDATION")
    print("="*80)

    validation = analyzer.validate_propositions()

    print("\nProposition 1 (Volatility Induction):")
    print(f"  ✓ Volatility Increase: {validation['proposition_1']['volatility_increase_pct']:.1f}%")
    print(f"  ✓ Alpha Increase: {validation['proposition_1']['alpha_increase_pct']:.1f}%")
    print(f"  ✓ Validated: {validation['proposition_1']['validated']}")

    print("\nProposition 2 (Information Cascades):")
    print(f"  ✓ Cascade Increase: {validation['proposition_2']['cascade_increase_pct']:.1f}%")
    print(f"  ✓ Validated: {validation['proposition_2']['validated']}")

    print("\nRegulatory Gap:")
    print(f"  ✓ Profit Reduction (Enforcement): {validation['regulatory_gap']['profit_reduction_pct']:.1f}%")
    print(f"  ✓ Cascade Reduction: {validation['regulatory_gap']['cascade_reduction_pct']:.1f}%")
    print(f"  ✓ Residual Alpha (Daily): ${validation['regulatory_gap']['residual_alpha_daily']:.2f}")
    print(f"  ✓ Insufficient Enforcement: {validation['regulatory_gap']['insufficient_enforcement']}")

    print("\n" + "="*80)

    print("="*80 + "\n")

    return engine, analyzer, validation


if __name__ == "__main__":
    engine, analyzer, validation = main()



ALGORITHMIC PREDATORY EQUILIBRIUM (APE) SIMULATION
Publication-Ready Code

Configuration:
  Episodes: 100
  Steps/Episode: 250
  Loss Aversion (λ): 2.25
  Predators: 5
  Prey: 95

Running Scenario A
Episode  10 | Predator Profit: $    0.00 | Volatility:  0.32%
Episode  20 | Predator Profit: $    0.00 | Volatility:  0.34%
Episode  30 | Predator Profit: $    0.00 | Volatility:  0.42%
Episode  40 | Predator Profit: $    0.00 | Volatility:  0.40%
Episode  50 | Predator Profit: $    0.00 | Volatility:  0.50%
Episode  60 | Predator Profit: $    0.00 | Volatility:  0.39%
Episode  70 | Predator Profit: $    0.00 | Volatility:  0.38%
Episode  80 | Predator Profit: $    0.00 | Volatility:  0.45%
Episode  90 | Predator Profit: $    0.00 | Volatility:  0.38%
Episode 100 | Predator Profit: $    0.00 | Volatility:  0.54%

Running Scenario B
Episode  10 | Predator Profit: $ -840.09 | Volatility:  0.33%
Episode  20 | Predator Profit: $ -329.85 | Volatility:  0.29%
Episode  30 | Predator Profit: $ -49

/tmp/ipython-input-1239012011.py:841: RuntimeWarning: divide by zero encountered in scalar divide
  alpha_increase = (summary_b['mean_predator_daily_profit'] - summary_a['mean_predator_daily_profit']) / summary_a['mean_predator_daily_profit']


In [3]:
import plotly.graph_objects as go
import plotly.express as px

# Ensure the main function has been run and 'analyzer' is available
# If not, run it:
# engine, analyzer, validation = main()

def plot_predator_profit(analyzer):
    fig = go.Figure()

    for scenario, results in analyzer.simulator.results.items():
        profits = [m.predator_profit for m in results]
        episodes = [m.episode_number for m in results]
        fig.add_trace(go.Scatter(
            x=episodes,
            y=profits,
            mode='lines',
            name=f'Scenario {scenario} Predator Profit'
        ))

    fig.update_layout(
        title='Predator Profit Across Scenarios (Interactive)',
        xaxis_title='Episode Number',
        yaxis_title='Predator Profit',
        hovermode='x unified'
    )
    fig.show()

def plot_market_volatility(analyzer):
    fig = go.Figure()

    for scenario, results in analyzer.simulator.results.items():
        volatilities = [m.market_volatility for m in results]
        episodes = [m.episode_number for m in results]
        fig.add_trace(go.Scatter(
            x=episodes,
            y=volatilities,
            mode='lines',
            name=f'Scenario {scenario} Market Volatility'
        ))

    fig.update_layout(
        title='Market Volatility Across Scenarios (Interactive)',
        xaxis_title='Episode Number',
        yaxis_title='Market Volatility',
        hovermode='x unified'
    )
    fig.show()


# Call the plotting functions
if 'analyzer' in locals():
    plot_predator_profit(analyzer)
    plot_market_volatility(analyzer)
else:
    print("Please run the main simulation first to get the 'analyzer' object.")

In [4]:
def plot_prey_loss(analyzer):
    fig = go.Figure()

    for scenario, results in analyzer.simulator.results.items():
        prey_losses = [-m.prey_loss for m in results] # Negative of prey_loss to show profit/loss from prey perspective
        episodes = [m.episode_number for m in results]
        fig.add_trace(go.Scatter(
            x=episodes,
            y=prey_losses,
            mode='lines',
            name=f'Scenario {scenario} Prey PnL'
        ))

    fig.update_layout(
        title='Prey Profit/Loss Across Scenarios (Interactive)',
        xaxis_title='Episode Number',
        yaxis_title='Prey Profit/Loss',
        hovermode='x unified'
    )
    fig.show()

# Call the new plotting function
if 'analyzer' in locals():
    plot_prey_loss(analyzer)
else:
    print("Please run the main simulation first to get the 'analyzer' object.")

### Simulation Results Analysis

The simulation aimed to model an Algorithmic Predatory Equilibrium (APE) across three scenarios: rational agents (A), MARL predators with behavioral prey (B), and regulated predatory behavior (C).

**Key Findings from the Results Table:**

| Metric                   | Scenario A (Rational) | Scenario B (Predatory) | Scenario C (Regulated) |
|:-------------------------|----------------------:|-----------------------:|-----------------------:|
| Predator Daily Profit    |                  0.00 |              -695.94 |             -3440.19 |
| Predator Total Profit    |                  0.00 |            -69593.72 |            -344018.71 |
| Prey Daily Loss          |               31278.77 |             34898.31 |             34721.81 |
| Prey Total Loss          |             3127877.10 |           3489831.06 |           3472180.89 |
| Mean Volatility          |                  0.0041 |                 0.0037 |                 0.0041 |
| Total Cascades           |               24995.00 |             24922.00 |             24861.00 |

**Interpretation:**

1.  **Predator Profit:**
    *   In Scenario A (Rational), predators show zero profit, as expected, because there are no behavioral anomalies to exploit. They are not actually 'predators' in this scenario, simply other rational agents.
    *   Surprisingly, in Scenario B (Predatory) and Scenario C (Regulated), predators exhibit significant *losses* rather than profits. This is contrary to the initial expectation that predators would profit from behavioral prey. The simulation's current parametrization seems to result in predators losing money even when attempting to exploit the market. This suggests that the simplified policy gradient for predators might not be effective enough to generate profits under the current market conditions and agent behaviors, or that the costs associated with trading (e.g., gas costs, market impact) outweigh potential gains.

2.  **Prey Loss:**
    *   Prey agents consistently incur losses across all scenarios. Their losses are substantial, in the order of millions over the total simulation steps.
    *   The difference in prey losses between Scenario A and Scenario B/C is relatively small. This implies that while predators are losing money, the behavioral aspects of prey (loss aversion, panic selling) still contribute to their overall losses, regardless of whether predators are actively exploiting them effectively or not.

3.  **Market Volatility:**
    *   Mean market volatility remains low and relatively consistent across all three scenarios (around 0.4%).
    *   This is another unexpected outcome. Proposition 1 posits that predators should *increase* volatility. The current results show a slight *decrease* in volatility in Scenario B compared to A, and then a return to Scenario A levels in Scenario C. This contradicts the hypothesis that MARL agents induce volatility.

4.  **Information Cascades:**
    *   The 'Total Cascades' metric, representing panic-selling events, is very high across all scenarios (around 25,000 events).
    *   Similar to volatility, there is no significant increase in cascades from Scenario A to B, and only a very minor reduction in C. This contradicts Proposition 2, which states that predatory agents induce more information cascades.

**Proposition Validation:**

*   **Proposition 1 (Volatility Induction):**
    *   `Volatility Increase: -10.3%` (Expected positive increase)
    *   `Alpha Increase: -inf%` (Expected positive increase, but predators are losing money)
    *   `Validated: False`
    *   The simulation results do **not** validate Proposition 1. Predators are not effectively inducing volatility or generating alpha (profit) from their strategies under the current parameters.

*   **Proposition 2 (Information Cascades):**
    *   `Cascade Increase: -0.3%` (Expected positive increase)
    *   `Validated: False`
    *   The simulation results do **not** validate Proposition 2. There is no observed increase in information cascades due to the presence of predatory agents.

*   **Regulatory Gap:**
    *   `Profit Reduction (Enforcement): -394.3%` (Indicates predators lost even more money in regulated scenario)
    *   `Cascade Reduction: 0.2%` (Minimal reduction)
    *   `Residual Alpha (Daily): $-3440.19` (Still losing money)
    *   `Insufficient Enforcement: False`
    *   The regulatory intervention in Scenario C seems to have exacerbated predator losses even further, rather than simply reducing their predatory profits. This is likely due to the fine mechanism affecting agents who are already struggling to profit. The enforcement is deemed 'insufficient' (returning `False` for `insufficient_enforcement`) because predators are not generating positive alpha to begin with, hence the regulation isn't curbing positive predatory behavior.

**Conclusion:**

The current simulation parameters and agent logic do not validate the core propositions of Algorithmic Predatory Equilibrium. The predators are not profitable and do not appear to induce significant volatility or information cascades. The behavioral prey exhibit consistent losses, but these are not demonstrably amplified by the presence of the current predator model. Further investigation into agent behaviors, policy learning, and market dynamics within the simulation is warranted to align with the theoretical propositions.